# QA問答系統
本專案範例採用OpenAI API，使用GPT模型與Embedding模型，建立一個使用自己資料庫的問答系統。

## 使用原始GPT模型回答問題

### 設定模型版本與API密鑰

In [1]:
import openai
EMBEDDING_MODEL = "text-embedding-ada-002" # 嵌入模型版本，使用目前最新的版本
GPT_MODEL = "gpt-3.5-turbo" # GPT模型版本，根據需要，你可以換成 gpt-4 以獲得更好的效果
openai.api_key = "sk-x1aAfxxxxxxxxxxxx" # <- 需要替換成你自己的API

### 設定問題後，直接讓GPT回答

In [2]:
question = "我想請問一下，我的機車是150cc的，需要交牌照稅嗎？"

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': '你是一個台北市的稅務局員工，協助民眾回答問題'},
        {'role': 'user', 'content': question},
    ],
    model=GPT_MODEL,
    temperature=0,
)
print(response['choices'][0]['message']['content'])

您好！根據台灣的相關法規，150cc以下的機車是需要交納牌照稅的。牌照稅的金額會根據您的機車排氣量和車齡來計算，您可以到您所在地的稅務局或是網路上的相關稅務資訊網站查詢具體的金額。如果您有任何其他稅務相關問題，歡迎再向我詢問！


### 先提供資訊，再讓GPT回答

In [3]:
info = """
經過搜尋後，資料庫中相關的問答資料如下(不同問題之間使用`---`隔開)：

Q: 供身心障礙者使用之車輛如何免徵使用牌照稅？應檢附之證件？有無車輛數及免稅額之限制？
A: "（一）身心障礙者自有車輛，不論是否有駕照，稅捐處都將主動核定免稅，車主免申請。
（二）身心障礙者因身心障礙情況，致無駕駛執照者，其配偶或同一戶籍二親等以內親屬所有供其使用之車輛，可申請免稅，申請時應檢附身心障礙證明。
（三）身心障礙免徵使用牌照稅每一身心障礙者以一輛為限，且免稅金額以2,400cc之稅額11,230為限，超過之部分，不予免徵，仍應繳納使用牌照稅。"
---
Q: 機車是否須繳納使用牌照稅？
A: （一）在150cc(含)以下者，不必繳納使用牌照稅。
（二）在151cc(含)以上者，要依汽缸總排氣量之多寡繳納使用牌照稅。
---
Q: 臺北市完全以電能為動力之汽車及機車是否有免徵使用牌照稅？
A: 本市完全以電能為動力之汽車及機車至110年12月31日止免徵使用牌照稅。
---
"""

In [4]:
question = "我想請問一下，我的機車是150cc的，需要交牌照稅嗎？"

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': '你是一個台北市的稅務局員工，協助民眾回答問題，並提供資訊'},
        {"role": "assistant", "content": info},
        {'role': 'user', 'content': question},
    ],
    model=GPT_MODEL,
    temperature=0,
)
print(response['choices'][0]['message']['content'])

根據資料庫中的資訊，150cc及以下的機車不需要繳納使用牌照稅，所以您的機車不需要交牌照稅。請注意，這些資訊僅供參考，若有任何疑問，建議您直接向稅務局詢問以獲得最準確的答案。


## 提供問答資料庫


### 準備資料
這邊我們以[臺北市稅捐稽徵處常見問答集](https://data.taipei/dataset/detail?id=2ab479d3-a115-431f-92de-a1a5c5dea5a0)為例

In [5]:
import pandas as pd
df = pd.read_csv("QA_dataset.csv")
df

,序號,問題,答案
0,1,可以到哪些稅務機關查詢被繼承人金融遺產呢？,（一）自109年7月1日起民眾在辦理遺產稅申報前，可直接前往全國任一地區國稅局，除了查調被繼...
1,2,不動產移轉，如何利用網路申報並辦理線上查欠？,民眾透過地方稅網路申報系統申報不動產移轉，並以電子檔傳送、郵寄、傳真或臨櫃遞送蓋妥義務人、權...
2,3,受疫情影響可以申請延分期嗎？如何申請？,因應嚴重特殊傳染性肺炎疫情，本處已訂定地方稅延期或分期繳納處理措施，個人及營利事業經中央目的...
3,4,便利超商繳稅限額?只能使用現金繳納嗎?,財政部自109年9月16日起，除提高便利超商繳稅限額為每筆新臺幣3萬元外，並開放使用「實體信...
4,5,可以使用非本人之信用卡繳稅嗎？,自109年9月16日起開放非本人信用卡繳稅，但採網際網路申報自繳稅款除外。
...,...,...,...
79,80,申請課稅明細跟繳納證明可否先預約申請再取件？可否跨區預約？,本處提供查調本市課稅明細及繳納證明等25項預約取件服務，申請人可透過網路、傳真或電話等方式選...
80,81,如何透過「稅務e申辦」申請簡易稅務案件？,本處網站建置「稅務e申辦」提供納稅義務人上傳身分證明文件影像檔，即可申辦及查詢簡易稅務案件，...
81,82,可否線上即時查詢、試算臺北市房屋稅或地價稅稅額？,至本處網站登入「房地稅額e化查詢平台」，憑自然人憑證、工商憑證或已註冊之健保卡，即可線上查詢...
82,83,使用臺北市稅捐稽徵處網站房地稅額e化查詢平台之稅額查詢功能無法順利用自然人憑證或健保卡登入？,基於資安考量，稅額查詢功能僅支援以IE瀏覽器登入，若確認已使用IE瀏覽器開啟該網頁，請參考網...


### 使用Embedding模型取得文字嵌入向量

In [6]:
# 定義一個函數，用來取得文字的嵌入向量
from tenacity import retry, stop_after_attempt # 避免因為網路問題而造成的錯誤，我們使用tenacity套件來自動重試

@retry(stop=stop_after_attempt(3))
def get_embedding(text: str):
    return openai.Embedding.create(input=[text], model=EMBEDDING_MODEL)["data"][0]["embedding"]

In [7]:
get_embedding("我們的文字會被轉換成一個1536維的向量")

[-0.011746440082788467,
 -0.010273970663547516,
 -0.013298861682415009,
 -0.001798944198526442,
 0.002200375311076641,
 0.009407811798155308,
 -0.014444855973124504,
 0.0006088093505240977,
 -0.021054310724139214,
 -0.02367943711578846,
 -0.005546744912862778,
 0.04418740049004555,
 -0.0034912843257188797,
 -0.01656361296772957,
 -0.0031981230713427067,
 0.009834228083491325,
 0.012539307586848736,
 0.001532434020191431,
 0.019721759483218193,
 -0.014831296168267727,
 0.016390381380915642,
 0.022546768188476562,
 0.007155800238251686,
 0.0006962580373510718,
 -0.034486427903175354,
 0.006806005723774433,
 0.0056200348772108555,
 -0.010260645300149918,
 -0.011486591771245003,
 0.0006775190704502165,
 0.01369862724095583,
 0.011100152507424355,
 -0.018389208242297173,
 -0.03102179430425167,
 -0.027637112885713577,
 -0.0017523049609735608,
 0.01058045681566,
 -0.0215207040309906,
 0.00804860983043909,
 -0.020507965236902237,
 0.017243213951587677,
 0.0018572433618828654,
 0.00616304995492

### 將所有問題轉換成嵌入向量
在事前準備中，我們使用Embedding模型將所有資料庫中的問題都轉換成嵌入向量，以方便後續的搜尋比較使用。

In [8]:
df["embedding"] = df["問題"].apply(get_embedding)
df

,序號,問題,答案,embedding
0,1,可以到哪些稅務機關查詢被繼承人金融遺產呢？,（一）自109年7月1日起民眾在辦理遺產稅申報前，可直接前往全國任一地區國稅局，除了查調被繼...,"[-0.0011391822481527925, -0.02377127669751644,..."
1,2,不動產移轉，如何利用網路申報並辦理線上查欠？,民眾透過地方稅網路申報系統申報不動產移轉，並以電子檔傳送、郵寄、傳真或臨櫃遞送蓋妥義務人、權...,"[-0.023056576028466225, -0.010698308236896992,..."
2,3,受疫情影響可以申請延分期嗎？如何申請？,因應嚴重特殊傳染性肺炎疫情，本處已訂定地方稅延期或分期繳納處理措施，個人及營利事業經中央目的...,"[-0.008783766999840736, -0.01474608201533556, ..."
3,4,便利超商繳稅限額?只能使用現金繳納嗎?,財政部自109年9月16日起，除提高便利超商繳稅限額為每筆新臺幣3萬元外，並開放使用「實體信...,"[-0.009257040917873383, -0.009587167762219906,..."
4,5,可以使用非本人之信用卡繳稅嗎？,自109年9月16日起開放非本人信用卡繳稅，但採網際網路申報自繳稅款除外。,"[-0.028891928493976593, -0.01572682149708271, ..."
...,...,...,...,...
79,80,申請課稅明細跟繳納證明可否先預約申請再取件？可否跨區預約？,本處提供查調本市課稅明細及繳納證明等25項預約取件服務，申請人可透過網路、傳真或電話等方式選...,"[0.013301858678460121, -0.01667880080640316, -..."
80,81,如何透過「稅務e申辦」申請簡易稅務案件？,本處網站建置「稅務e申辦」提供納稅義務人上傳身分證明文件影像檔，即可申辦及查詢簡易稅務案件，...,"[0.0017664377810433507, -0.006662579253315926,..."
81,82,可否線上即時查詢、試算臺北市房屋稅或地價稅稅額？,至本處網站登入「房地稅額e化查詢平台」，憑自然人憑證、工商憑證或已註冊之健保卡，即可線上查詢...,"[-0.0005041414988227189, -0.01986890845000744,..."
82,83,使用臺北市稅捐稽徵處網站房地稅額e化查詢平台之稅額查詢功能無法順利用自然人憑證或健保卡登入？,基於資安考量，稅額查詢功能僅支援以IE瀏覽器登入，若確認已使用IE瀏覽器開啟該網頁，請參考網...,"[-0.00983906164765358, -0.02059950679540634, -..."


## 對提出的問題，找出最相似的題目作為參考

### 先計算問題的嵌入向量
+ 和所有已經計算好嵌入向量的問題做相似度比較(使用餘弦相似度)
+ 找出相似度最高的5個問題

In [9]:
# 計算並找出最相似的問題
from scipy import spatial
question = "我想請問一下，我的機車是150cc的，需要交牌照稅嗎？"
embedding = get_embedding(question)
df["相似度"] = df["embedding"].apply(lambda x: 1 - spatial.distance.cosine(x, embedding))
df = df.sort_values(by="相似度", ascending=False)
df.head(5)

,序號,問題,答案,embedding,相似度
48,49,機車是否須繳納使用牌照稅？,（一）在150cc(含)以下者，不必繳納使用牌照稅。\n（二）在151cc(含)以上者，要依...,"[-0.004087657202035189, -0.00985042005777359, ...",0.915610
47,48,供身心障礙者使用之車輛如何免徵使用牌照稅？應檢附之證件？有無車輛數及免稅額之限制？,（一）身心障礙者自有車輛，不論是否有駕照，稅捐處都將主動核定免稅，車主免申請。\n（二）身心...,"[0.009465841576457024, -0.0019561201333999634,...",0.875463
49,50,車輛欠繳使用牌照稅行駛公共道路，有處罰規定嗎？,逾期未完稅的交通工具，在滯納期滿後使用公共道路經查獲者，除責令補稅外，處以應納稅額1倍以下之罰鍰。,"[-0.008359411731362343, 0.006033661309629679, ...",0.864001
50,51,臺北市完全以電能為動力之汽車及機車是否有免徵使用牌照稅？,本市完全以電能為動力之汽車及機車至110年12月31日止免徵使用牌照稅。,"[0.0013043637154623866, -0.01299822423607111, ...",0.862671
6,7,如果?有收到當年度定期開徵之使用牌照稅、房屋稅或地價稅稅單，是不是只能使用自然人憑證/工商憑...,您可持自然人憑證/工商憑證、已註冊之健保卡或已註冊內政部行動身分識別(TAIWAN FidO...,"[0.0016425909707322717, -0.014337744563817978,...",0.848921


### 將相似度最高的問題，作為回答的依據

In [10]:
# 將最相似的問題加入到預先提供的資訊中
question_list = df["問題"].head(5).tolist()
answer_list = df["答案"].head(5).tolist()

info = """
經過搜尋後，資料庫中相關的問答資料如下(不同問題之間使用`---`隔開)：
---
"""

for q, a in zip(question_list, answer_list):
    info += f"Q: {q}\nA: {a}\n---\n"

print(info)


經過搜尋後，資料庫中相關的問答資料如下(不同問題之間使用`---`隔開)：
---
Q: 機車是否須繳納使用牌照稅？
A: （一）在150cc(含)以下者，不必繳納使用牌照稅。
（二）在151cc(含)以上者，要依汽缸總排氣量之多寡繳納使用牌照稅。
---
Q: 供身心障礙者使用之車輛如何免徵使用牌照稅？應檢附之證件？有無車輛數及免稅額之限制？
A: （一）身心障礙者自有車輛，不論是否有駕照，稅捐處都將主動核定免稅，車主免申請。
（二）身心障礙者因身心障礙情況，致無駕駛執照者，其配偶或同一戶籍二親等以內親屬所有供其使用之車輛，可申請免稅，申請時應檢附身心障礙證明。
（三）身心障礙免徵使用牌照稅每一身心障礙者以一輛為限，且免稅金額以2,400cc之稅額11,230為限，超過之部分，不予免徵，仍應繳納使用牌照稅。
---
Q: 車輛欠繳使用牌照稅行駛公共道路，有處罰規定嗎？
A: 逾期未完稅的交通工具，在滯納期滿後使用公共道路經查獲者，除責令補稅外，處以應納稅額1倍以下之罰鍰。
---
Q: 臺北市完全以電能為動力之汽車及機車是否有免徵使用牌照稅？
A: 本市完全以電能為動力之汽車及機車至110年12月31日止免徵使用牌照稅。
---
Q: 如果?有收到當年度定期開徵之使用牌照稅、房屋稅或地價稅稅單，是不是只能使用自然人憑證/工商憑證才能在便利商店補單繳納稅款？
A: 您可持自然人憑證/工商憑證、已註冊之健保卡或已註冊內政部行動身分識別(TAIWAN FidO)之行動裝置（於開徵起日前5個日曆天至開徵迄日屆滿後2日23時）至四大便利商店(統一、全家、萊爾富及來來OK）之多媒體機kiosk進行查詢當年度定期開徵之稅目資訊，勾選列印繳納單並當日至店家櫃檯掃描條碼完成繳稅(使用牌照稅只需輸入納稅義務人身分證號碼/統一編號及車牌號碼，即可列印繳款條碼於店家櫃檯完成繳稅)，每筆金額限新臺幣3萬元以下(自109年9月16日起)，自110年起民眾如需繳納證明請向本處所屬分處申請。
---



### 根據參考資訊回答問題

In [11]:
response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': '你是一個台北市的稅務局員工，協助民眾回答問題，並提供資訊'},
        {"role": "assistant", "content": info},
        {'role': 'user', 'content': question},
    ],
    model=GPT_MODEL,
    temperature=0,
)
print(response['choices'][0]['message']['content'])

根據資料庫中的資訊，150cc及以下的機車不需要繳納使用牌照稅，所以您的機車不需要交牌照稅。如果您有其他問題，歡迎再提出。


## 補充：API費用試算
提供本範例使用OpenAI API所產生的用量及費用供讀者參考：
+ `gpt-3.5-turbo-0613`
    + 783 tokens(661 prompt + 122 completion)
    + 當時費率為 
        + Input: $0.0015 / 1K tokens
        + Output: $0.002 / 1K tokens
    + 約 0.0012355 美元
+ `text-embedding-ada-002-v2`
    + 2935 tokens
    + 當時費率為
        + $0.0001 / 1K tokens
    + 約0.0002935 美元
+ 小記共 
    + `0.001529` 美元
    + 約 `0.05` 元新台幣不到